In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns
import matplotlib
from tqdm.notebook import tqdm 
from matplotlib import rc
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

rc("font", **{"family": "sans-serif", "sans-serif": "DejaVu Sans"})
rc("figure", **{"dpi": 200})
rc(
    "axes",
    **{"spines.right": False, "spines.top": False, "xmargin": 0.0, "ymargin": 0.05}
)

# Load data

In [ ]:
df = pd.read_csv('../input/eurecom-aml-2022-challenge-1/public/train.csv', low_memory=True)
df_test = pd.read_csv('../input/eurecom-aml-2022-challenge-1/public/test_feat.csv', low_memory=True)

In [ ]:
print("Training dataset size: ", len(df))
print("Testing dataset size: ", len(df_test))

# Data analysis


In [ ]:
"""
pd.options.display.max_columns = df.shape[1]
df.head()
"""

All features are numerical and continuous.

In [ ]:
"""
num_duplicate = len(df) - df.duplicated().value_counts()
print("Number of duplicates: ", num_duplicate)

num_nan = df.isnull().sum().sum()
print("Number of missing values: ", num_nan)
df_without_nan = df.dropna()
print("Number of columns containing missing values: ", len(df)-len(df_without_nan)) 
"""

There is no duplicate rows in the dataset. But there is more than 1M missing values among around 108,000 records.

In [ ]:
"""
# Plot mean and variance of the features
array_mean_std = df.describe().iloc[1:3, 1:].values

fig, ax1 = plt.subplots()

features_indexes = np.arange(0, 112)
ax1.set_xlabel('Features')

ax1.set_ylabel('Mean', color = 'b')
ax1.bar(features_indexes - 0.4, array_mean_std[0], width = 0.6, color = 'b')
ax1.set_ylim(top=4000, bottom=0)
ax1.tick_params(axis='y', labelcolor='b')

ax2 = ax1.twinx()
ax2.set_ylabel('Variance', color = 'g')
ax2.bar(features_indexes + 0.4, array_mean_std[1], width = 0.6, color = 'g')
ax2.set_ylim(top=800, bottom=0)
ax2.tick_params(axis='y', labelcolor='g')

fig.tight_layout()
plt.title("Mean and variance of the features")
plt.show()
"""

The distributions of the different features are really diverse as their means and variances vary a lot between each other. In addition, some features are outliers are they have small variance (< 0.5). In the preprocessing, we should remove outliers and standardize features if the scale is not meaningful for some models.

In [ ]:
"""
corr = df.corr() # Correlation matrix

# Heatmap showing correlation with target
plt.figure(figsize=(4, 8))
cmap = sns.diverging_palette(240, 10, n=9, as_cmap=True)
heatmap = sns.heatmap(corr[['fact_temperature']].sort_values(by='fact_temperature', ascending=False),vmin=-1, vmax=1, annot=False, cmap=cmap)
heatmap.set_title('Features Correlating with Temperature')
"""

Many features that have negligible values for correlation with the temperature. To train and test our models we may keep only features with high correlation with fact_temperature.

### Distribution of train and test points in the dataset

In [ ]:
"""
tr_coordinates = df[['fact_latitude','fact_longitude']].drop_duplicates()
te_coordinates = df_test[['fact_latitude','fact_longitude']].drop_duplicates()
"""

In [ ]:
"""
merged = te_coordinates.merge(tr_coordinates, how='left', indicator=True)
tropic_coordinates = merged[merged['_merge']=='left_only']
common_coordinates = merged[merged['_merge']=='both']
"""

In [ ]:
"""
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=[10, 3], dpi=200, sharex=True, sharey=True)
ax0.scatter(tr_coordinates['fact_longitude'], tr_coordinates['fact_latitude'], 
            s=1, c='tab:green', label='Train data')
ax1.scatter(common_coordinates['fact_longitude'], common_coordinates['fact_latitude'], 
            s=1, c='tab:orange', label='In-domain test data')
ax1.scatter(tropic_coordinates['fact_longitude'], tropic_coordinates['fact_latitude'], 
            s=1, c='tab:red', label='Out-domain test data')
ax0.legend(), ax1.legend()
fig.suptitle('Distribution of train and test points in the dataset', y=1.01, fontsize=14)
plt.show()
"""


# Data Preprocessing


### Removing outliers

In [ ]:
df_without_outlier = df.loc[:, df.std()>0.5]

In [ ]:
num_nan_new = df_without_outlier.isnull().sum().sum()
print("Number of missing values: ", num_nan_new)
df_without_nan_new = df_without_outlier.dropna()
print("Number of columns containing missing values: ", len(df_without_outlier)-len(df_without_nan_new)) 

### Handling missing values in the dataset

In [ ]:
def preprocess_nan(df, mode='naive'):
    # remove the rows with nan 
    if mode == 'naive':
        df_tr = df.dropna()
        X = df_tr.iloc[:, 1:-1].values
        y = df_tr.iloc[:, -1].values
    # replace nan with the mean of the corresponding column
    elif mode == 'mean':
        column_means = df.mean(axis=0)
        df_tr = df.fillna(column_means, axis=0)
        X = df_tr.iloc[:, 1:-1].values
        y = df_tr.iloc[:, -1].values
    return X, y, df_tr

In [ ]:
X, y, df_tr = preprocess_nan(df_without_outlier, mode='naive') # mode = 'naive' or "mean"

### Standardising data

In [ ]:
X_scale = StandardScaler().fit_transform(X)

### Reducing the input space

#### By dimensionality reduction

In [ ]:
"""
# Plot number of components over the cumulative explained variance
pca = PCA().fit(X_scale)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');
plt.title('Fig.2: Number of components over the cumulative explained variance');
"""

In [ ]:
pca = PCA(n_components=30)
X_pca = pca.fit_transform(X_scale)
print("Explained varaiance for 30 principal components: ", np.sum(pca.explained_variance_ratio_))

#### By feature selection

In [ ]:
"""
def select_features(X_train, y_train):
    fs = SelectKBest(score_func=mutual_info_regression, k=30)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    return X_train_fs, fs
"""

In [ ]:
"""
df_samples = df_tr.sample(n = 100000) # select samples randomly (instead of using all the training dataset to perform the feature selection)
X_samples = df_samples.iloc[:, 1:-1].values
X_samples_scale = StandardScaler().fit_transform(X_samples)
y_samples = df_samples.iloc[:, -1].values

X_train_fs, fs = select_features(X_samples_scale, y_samples)

# plot the scores
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()
"""

In [ ]:
"""
# Get top 30 most important features 
mask = fs.get_support()
new_features = [] 
feature_names = list((df_tr.drop(['fact_temperature'],axis=1)).columns.values)
for bool, feature in zip(mask, feature_names):
    if bool:
        new_features.append(feature)

# Select these features for X_fs
col_selection_fs = new_features
X_fs = df_tr[col_selection_fs].values
"""

## Preprocessing of the testing dataset

In [ ]:
# Apply same pre-processing on test dataset
col_selection = df_tr.columns[:-1]
df_test_new = df_test[col_selection]
X_test = df_test_new.iloc[:, 1:].values

scaler = StandardScaler()
scaler.fit(X)
X_test_scale = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scale)


# Model Selection

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scale, y, random_state=1, test_size=0.9)

In [ ]:
ystd = 1.0
def compute_rmse(y, ypred, ystd=1.):
    return np.mean((y - ypred)**2)**0.5 * ystd

In [ ]:
############ Try different models ############
model = Lasso(alpha=0.1)  # complexity 0(n)
# model = Ridge(alpha=0.1)  # complexity 0(n)
# model = RandomForestRegressor(max_depth=10, min_samples_leaf=5, n_estimators=10)
# model = MLPRegressor(max_iter=30, validation_fraction=0.2, early_stopping=True, tol=1e-3, n_iter_no_change=5,
#                    hidden_layer_sizes=(300, 100, 100, ), learning_rate_init=1e-3, random_state=0, verbose=1)     # complexity 0(n*n1 + n1*n2* + ...) ~= O(Kn)
###############################################

model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
print(f'Train RMSE: {compute_rmse(y_train, y_train_pred, ystd=1.0):.3f}')
print(f'Valid RMSE: {compute_rmse(y_val, y_val_pred, ystd=1.0):.3f}')

# Parameter Optimisation

Note: Skip this part is no optimisation is made with grid search.

In [ ]:
def get_cv_idx(n, test_size=0.2, n_splits=5):
    train_idx, test_idx = [], []
    for _ in range(n_splits):
        idx = np.random.permutation(n)
        train_size = int(n * (1 - test_size)) if isinstance(test_size, float) else n - test_size
        train_idx.append(idx[:train_size])
        test_idx.append(idx[train_size:])
    return train_idx, test_idx

In [ ]:
train_idx, cv_idx = get_cv_idx(len(X_train), test_size=0.9, n_splits=5)

In [ ]:
########## grid search parameters ########## 
param_grid_lasso = {
    "alpha": [0.001, 0.01, 0.1, 1.0]
}

param_grid_ridge = {
    "alpha": [0.01, 0.1, 1.0, 10.0]
}

param_grid_rf = {
   'max_depth' : [5, 8, 10], 'min_samples_leaf': [10, 50, 100],'n_estimators': [200, 250, 300]
}
########## grid search parameters ########## 

best_model = GridSearchCV(model, 
                      param_grid_lasso, # TO MODIFY
                      n_jobs=-1, 
                      verbose=1,
                      cv=zip(train_idx, cv_idx), 
                      scoring='neg_root_mean_squared_error').fit(X_train, y_train)
print('Done!')

In [ ]:
print("Best parameters set found on cv set:")
print(best_model.best_params_)
print()
print("Grid scores on cv set:")
means = best_model.cv_results_["mean_test_score"]
stds = best_model.cv_results_["std_test_score"]
for mean, std, params in zip(means, stds, best_model.cv_results_["params"]):
    print("%0.3f (+/-%0.03f) for %r" % (-mean * ystd, (std * ystd) * 2, params))
print()
print("Error on the validation set")
y_val_pred = best_model.predict(X_val)
print(f'Valid RMSE: {compute_rmse(y_val, y_val_pred, ystd=1.0):.3f}')

# Model Evaluation

In [ ]:
y_test_pred = best_model.predict(X_test_pca)  # If parameter optimisation using grid search
# y_test_pred = model.predict(X_test_pca) # If not using the "Parameter Optimisation" part


# Submission


In [ ]:
submission_df = pd.DataFrame(data={'index': df_test['index'].values,
                                   'fact_temperature': y_test_pred.squeeze()})
# Save the predictions into a csv file
# Notice that this file should be saved under the directory `/kaggle/working` 
# so that you can download it later
submission_df.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
# Check the submission file
! head -6 "/kaggle/working/submission.csv"